In [1]:
import tensorflow as tf

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
var = tf.Variable(initial_value=0.0)

In [3]:
sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(tf.global_variables_initializer())
sess2.run(tf.global_variables_initializer())

In [4]:
print("Initial value of var in session 1:", sess1.run(var))
print("Initial value of var in session 2:", sess2.run(var))

Initial value of var in session 1: 0.0
Initial value of var in session 2: 0.0


In [5]:
sess1.run(var.assign_add(1.0))
print("Incremented var in session 1")

Incremented var in session 1


In [6]:
print("Value of var in session 1:", sess1.run(var))
print("Value of var in session 2:", sess2.run(var))

Value of var in session 1: 1.0
Value of var in session 2: 0.0


In [7]:
sess1.close()
sess2.close()

In [8]:
tasks = ["localhost:2222", "localhost:2223"]
jobs = {"local": tasks}

In [9]:
cluster = tf.train.ClusterSpec(jobs)

In [10]:
server1 = tf.train.Server(cluster, job_name="local", task_index=0)

In [11]:
server2 = tf.train.Server(cluster, job_name="local", task_index=1)

In [12]:
print(server1)

In [13]:
server1.target

b'grpc://localhost:2222'

In [14]:
tf.reset_default_graph()

In [15]:
var = tf.Variable(initial_value=0.0, name='var')

In [16]:
sess1 = tf.Session(server1.target)
sess2 = tf.Session(server2.target)

In [17]:
sess1.run(tf.global_variables_initializer())

In [18]:
print("Initial value of var in session 1:", sess1.run(var))
print("Initial value of var in session 2:", sess2.run(var))

Initial value of var in session 1: 0.0
Initial value of var in session 2: 0.0


In [19]:
sess1.run(var.assign_add(1.0))
print("Incremented var in session 1")

Incremented var in session 1


In [20]:
print("Value of var in session 1:", sess1.run(var))
print("Value of var in session 2:", sess2.run(var))

Value of var in session 1: 1.0
Value of var in session 2: 1.0


In [21]:
def run_with_location_trace(sess, op):
    # From https://stackoverflow.com/a/41525764/7832197
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    sess.run(op, options=run_options, run_metadata=run_metadata)
    for device in run_metadata.step_stats.dev_stats:
      print(device.device)
      for node in device.node_stats:
        print("  ", node.node_name)

In [25]:
run_with_location_trace(sess1, var)

/job:local/replica:0/task:0/device:CPU:0
   _SOURCE
/job:local/replica:0/task:0/device:GPU:0
   _SOURCE
   var


In [26]:
run_with_location_trace(sess2, var)

/job:local/replica:0/task:1/device:CPU:0
   RecvTensor
/job:local/replica:0/task:1/device:CPU:0
   _SOURCE
/job:local/replica:0/task:0/device:GPU:0
   _SOURCE
   var


In [23]:
run_with_location_trace(sess1, var.assign_add(1.0))

/job:local/replica:0/task:0/device:CPU:0
   _SOURCE
/job:local/replica:0/task:0/device:GPU:0
   _SOURCE
   AssignAdd_1/value
   var
   AssignAdd_1


In [24]:
run_with_location_trace(sess2, var)

/job:local/replica:0/task:1/device:CPU:0
   RecvTensor
/job:local/replica:0/task:1/device:CPU:0
   _SOURCE
/job:local/replica:0/task:0/device:GPU:0
   _SOURCE
   var


In [30]:
run_with_location_trace(sess2, var.assign_add(1.0))

/job:local/replica:0/task:1/device:CPU:0
   RecvTensor
/job:local/replica:0/task:1/device:CPU:0
   _SOURCE
/job:local/replica:0/task:0/device:GPU:0
   _SOURCE
   AssignAdd_2/value
   var
   AssignAdd_2


In [28]:
a = tf.Variable(initial_value=2.0)
b = tf.Variable(initial_value=1.0)
c = a + b

In [29]:
sess1.run(tf.global_variables_initializer())

In [31]:
run_with_location_trace(sess2, c)

/job:local/replica:0/task:1/device:CPU:0
   RecvTensor
/job:local/replica:0/task:1/device:CPU:0
   _SOURCE
/job:local/replica:0/task:0/device:GPU:0
   _SOURCE
   Variable_1
   Variable
   Variable_1/read
   Variable/read
   add


In [32]:
with tf.device("/job:local/task:0"):
    var1 = tf.Variable(0.0, name='var1')
with tf.device("/job:local/task:1"):
    var2 = tf.Variable(0.0, name='var2')
    
sess1.run(tf.global_variables_initializer())

In [33]:
run_with_location_trace(sess1, var1)

/job:local/replica:0/task:0/device:CPU:0
   _SOURCE
/job:local/replica:0/task:0/device:GPU:0
   _SOURCE
   var1


In [34]:
run_with_location_trace(sess1, var2)

/job:local/replica:0/task:0/device:CPU:0
   RecvTensor
/job:local/replica:0/task:0/device:CPU:0
   _SOURCE
/job:local/replica:0/task:1/device:GPU:0
   _SOURCE
   var2


In [35]:
run_with_location_trace(sess2, var2)

/job:local/replica:0/task:1/device:GPU:0
   _SOURCE
   var2
/job:local/replica:0/task:1/device:CPU:0
   _SOURCE


In [36]:
run_with_location_trace(sess2, var1)

/job:local/replica:0/task:1/device:CPU:0
   RecvTensor
/job:local/replica:0/task:1/device:CPU:0
   _SOURCE
/job:local/replica:0/task:0/device:GPU:0
   _SOURCE
   var1


In [37]:
sess1.close()
sess2.close()

In [38]:
cluster = tf.train.ClusterSpec({"local": ["localhost:2224", "localhost:2225"]})
server1 = tf.train.Server(cluster, job_name="local", task_index=0)
server2 = tf.train.Server(cluster, job_name="local", task_index=1)

In [39]:
graph1 = tf.Graph()
with graph1.as_default():
    var1 = tf.Variable(0.0, name='var')
sess1 = tf.Session(target=server1.target, graph=graph1)
print(graph1.get_operations())

[<tf.Operation 'var/initial_value' type=Const>, <tf.Operation 'var' type=VariableV2>, <tf.Operation 'var/Assign' type=Assign>, <tf.Operation 'var/read' type=Identity>]


In [40]:
graph2 = tf.Graph()
sess2 = tf.Session(target=server2.target, graph=graph2)
print(graph2.get_operations())

[]


In [41]:
with graph2.as_default():
    var2 = tf.Variable(0.0, name='var')

In [42]:
sess1.run(var1.assign(2.0))
sess2.run(var2)

2.0

In [43]:
sess1.close()
sess2.close()
